<h2>Import packages and install histomics_detect</h2>

In [ ]:
# install histomics_detect
!pip install -e /tf/notebooks/histomics_detect

# install histomics_stream
!pip install -e /tf/notebooks/histomics_stream

# add to system path
import sys

sys.path.append("/tf/notebooks/histomics_detect/")
sys.path.append("/tf/notebooks/histomics_stream/")

In [ ]:
import os
import re
import numpy as np
import tensorflow as tf

# import dataset related packages
from histomics_detect.io import dataset
from histomics_detect.augmentation import crop, flip, jitter, shrink
from histomics_detect.visualization import plot_inference

# import whole-slide image handling pipeline
import histomics_stream as hs

number_epochs = 50  # Set to a number smaller than 50 for speed during debug

<h2>Define dataset parameters and create datasets - DCC example</h2>

In [ ]:
# input data path
path = "/tf/notebooks/DCC/data/"

# training parameters
train_tile = 224  # input image size
min_area_thresh = 0.5  # % of object area that must be in random crop to be included
width = tf.constant(train_tile, tf.int32)
height = tf.constant(train_tile, tf.int32)
min_area = tf.constant(min_area_thresh, tf.float32)

# split dataset into training and validation
cases = [
    "131458",
    "91315_leica_at2_40x",
    "135062",
    "93094",
    "131453",
    "131450",
    "135060",
    "131463",
    "131459",
    "131440",
    "131460",
    "93096",
    "131449",
    "131457",
    "131461",
    "93098",
    "131447",
    "93092",
    "131443",
    "93095",
    "131448",
    "93099",
    "91316_leica_at2_40x",
    "131462",
    "93091",
    "135065",
    "131446",
    "131441",
    "101626",
    "93093",
    "131454",
    "93097",
    "131445",
    "131444",
    "131456",
    "93090",
]
id = np.argsort(np.random.rand(len(cases) - 1))[0 : np.ceil(0.9 * len(cases)).astype(np.int32)]
training = [cases[i] for i in id]
validation = list(set(cases).difference(training))

# define parser for filenames
def parser(file):
    name = os.path.splitext(file)[0]
    case = name.split(".")[2]
    roi = ".".join([name.split(".")[1]] + name.split(".")[-3:])
    return case, roi


# generate training, validation datasets
ds_train_roi = dataset(path, parser, parser, train_tile, training)
ds_validation_roi = dataset(path, parser, parser, 0, validation)

# build training dataset
ds_train_roi = ds_train_roi.map(lambda x, y, z: (*crop(x, y, width, height, min_area_thresh), z))
ds_train_roi = ds_train_roi.map(lambda x, y, z: (*flip(x, y), z))
ds_train_roi = ds_train_roi.map(lambda x, y, z: (x, jitter(y, 0.05), z))
ds_train_roi = ds_train_roi.map(lambda x, y, z: (x, shrink(y, 0.05), z))
ds_train_roi = ds_train_roi.prefetch(tf.data.experimental.AUTOTUNE)

# build validation datasets
ds_validation_roi = ds_validation_roi.prefetch(tf.data.experimental.AUTOTUNE)

<h2>Create and train detection model - DCC example</h2>

In [ ]:
# import network generation and training packages
from histomics_detect.networks.rpns import rpn
from histomics_detect.models.faster_rcnn import FasterRCNN

# choices for anchor sizes - all anchors 1:1 aspect ratio
anchor_px = tf.constant(
    [32, 64, 96], dtype=tf.int32
)  # width/height of square anchors in pixels at input mag.

# feature network parameters
backbone_stride = 1  # strides in feature generation network convolution
backbone_blocks = 14  # number of residual blocks to use in backbone
backbone_dimension = 256  # number of features generated by rpn convolution

# rpn network parameters
rpn_kernel = [3]  # kernel size for rpn convolution
rpn_act_conv = ["relu"]  # activation for rpn convolutional layers

# anchor filtering parameters
neg_max = 128  # maximum number of negative/positive anchors to keep in each roi
pos_max = 128
rpn_lmbda = 10.0  # weighting for rpn regression loss
roialign_tiles = 3.0  # roialign - number of horizontal/vertical tiles in a proposal
roialing_pool = 2.0  # roialign - number of horizontal/vertical samples in each tile

# create backbone and rpn networks
resnet50 = tf.keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(train_tile, train_tile, 3),
    pooling=None,
)
rpnetwork, backbone = rpn(
    resnet50,
    n_anchors=tf.size(anchor_px),
    stride=backbone_stride,
    blocks=backbone_blocks,
    kernels=rpn_kernel,
    dimensions=[backbone_dimension],
    activations=rpn_act_conv,
)

# create FasterRCNN keras model
model = FasterRCNN(rpnetwork, backbone, [width, height], anchor_px, rpn_lmbda)

# compile FasterRCNN model with losses
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=[
        tf.keras.losses.BinaryCrossentropy(from_logits=True),
        tf.keras.losses.Huber(),
    ],
)

# fit FasterRCNN model
model.fit(
    x=ds_train_roi,
    batch_size=1,
    epochs=number_epochs,
    verbose=1,
    validation_data=ds_validation_roi,
    validation_freq=number_epochs,
)

<h2>Define dataset parameters and create datasets - DLBCL example</h2>

In [ ]:
# import dataset related packages
from histomics_detect.io import dataset, resize
from histomics_detect.augmentation import crop, flip, jitter, shrink
from histomics_detect.visualization import plot_inference
import numpy as np
import os

# input data path
path = "/tf/notebooks/DLBCL/detection/"

# training parameters
train_tile = 224  # input image size
min_area_thresh = 0.5  # % of object area that must be in crop to be included
width = tf.constant(train_tile, tf.int32)
height = tf.constant(train_tile, tf.int32)
min_area = tf.constant(min_area_thresh, tf.float32)

# define filename parsers
def png_parser(png):
    file = os.path.splitext(png)[0]
    case = file.split(".")[0]
    roi = ".".join(file.split(".")[1:])
    return case, roi


def csv_parser(csv):
    file = os.path.splitext(csv)[0]
    case = file.split(".")[0]
    roi = ".".join(file.split(".")[1:2] + file.split(".")[-3:])
    return case, roi


training = [
    "DCBT_2_CMYC",
    "DCBT_3_CMYC",
    "DCBT_5_CMYC",
    "DCBT_9_CMYC",
    "DCBT_10_CMYC",
    "DCBT_12_CMYC",
    "DCBT_14_CMYC",
    "DCBT_18_CMYC",
    "DCBT_19_CMYC",
    "DCBT_20_CMYC",
    "DCBT_21_CMYC",
    "DCBT_22_CMYC",
]
validation = [
    "DCBT_1_CMYC",
    "DCBT_4_CMYC",
    "DCBT_6_CMYC",
    "DCBT_8_CMYC",
    "DCBT_11_CMYC",
    "DCBT_13_CMYC",
    "DCBT_15_CMYC",
    "DCBT_16_CMYC",
    "DCBT_17_CMYC",
]


# generate training, validation datasets
ds_train_roi = dataset(path, png_parser, csv_parser, train_tile, training)
ds_validation_roi = dataset(path, png_parser, csv_parser, 0, validation)

# build training dataset
ds_train_roi = ds_train_roi.map(lambda x, y, z: (*resize(x, y, 2.0), z))
ds_train_roi = ds_train_roi.map(lambda x, y, z: (*crop(x, y, width, height, min_area_thresh), z))
ds_train_roi = ds_train_roi.map(lambda x, y, z: (*flip(x, y), z))
ds_train_roi = ds_train_roi.map(lambda x, y, z: (x, jitter(y, 0.05), z))
ds_train_roi = ds_train_roi.map(lambda x, y, z: (x, shrink(y, 0.05), z))
ds_train_roi = ds_train_roi.prefetch(tf.data.experimental.AUTOTUNE)

# build validation datasets
ds_validation_roi = ds_validation_roi.map(lambda x, y, z: (*resize(x, y, 2.0), z))
ds_validation_roi = ds_validation_roi.prefetch(tf.data.experimental.AUTOTUNE)

<h2>Create and train detection model - DLBCL example</h2>

In [ ]:
# import network generation and training packages
from histomics_detect.networks.rpns import rpn
from histomics_detect.models.faster_rcnn import FasterRCNN

# choices for anchor sizes - all anchors 1:1 aspect ratio
anchor_px = tf.constant(
    [32, 48, 64], dtype=tf.int32
)  # width/height of square anchors in pixels at input mag.

# feature network parameters
backbone_stride = 1  # strides in feature generation network convolution
backbone_blocks = 14  # number of residual blocks to use in backbone
backbone_dimension = 256  # number of features generated by rpn convolution

# rpn network parameters
rpn_kernel = [3]  # kernel size for rpn convolution
rpn_act_conv = ["relu"]  # activation for rpn convolutional layers

# anchor filtering parameters
neg_max = 128  # maximum number of negative/positive anchors to keep in each roi
pos_max = 128
rpn_lmbda = 10.0  # weighting for rpn regression loss
roialign_tiles = 3.0  # roialign - number of horizontal/vertical tiles in a proposal
roialing_pool = 2.0  # roialign - number of horizontal/vertical samples in each tile

# create backbone and rpn networks
resnet50 = tf.keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(train_tile, train_tile, 3),
    pooling=None,
)
rpnetwork, backbone = rpn(
    resnet50,
    n_anchors=tf.size(anchor_px),
    stride=backbone_stride,
    blocks=backbone_blocks,
    kernels=rpn_kernel,
    dimensions=[backbone_dimension],
    activations=rpn_act_conv,
)

# create FasterRCNN keras model
model = FasterRCNN(rpnetwork, backbone, [width, height], anchor_px, rpn_lmbda)

# compile FasterRCNN model with losses
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=[
        tf.keras.losses.BinaryCrossentropy(from_logits=True),
        tf.keras.losses.Huber(),
    ],
)

# fit FasterRCNN model
model.fit(
    x=ds_train_roi,
    batch_size=1,
    epochs=number_epochs,
    verbose=1,
    validation_data=ds_validation_roi,
    validation_freq=number_epochs,
)

<h2>Inference on a single image - model.call() </h2>

In [ ]:
# generate and visualize thresholded, roialign outputs
data = ds_validation_roi.shuffle(100).take(1).get_single_element()
rgb = tf.cast(data[0], tf.uint8)
regressions = model(rgb, tau=0.5, nms_iou=0.3)
plot_inference(rgb, regressions)

<h2>Raw inference on a single image - model.raw() </h2>

In [ ]:
# generate raw rpn outputs
objectness, boxes, features = model.raw(rgb)

# threshold rpn proposals
boxes_positive, objectness_positive, positive = model.threshold(boxes, objectness, model.tau)

# perform non-max suppression on rpn positive predictions
boxes_nms, objectness_nms, selected = model.nms(boxes_positive, objectness_positive, model.nms_iou)

# generate roialign predictions for rpn positive predictions
align_boxes = model.align(boxes_nms, features, model.field, model.pool, model.tiles)

# apply thresholding, nms, and roialign
plot_inference(rgb, align_boxes)

<h2>Batch inference using tf.data.Dataset.map </h2>

In [ ]:
# mapping model using data.Dataset.map keeps outputs from different images separate
map_output = ds_validation_roi.take(5).map(lambda x, y, z: (model(x), y, z))
map_output = [element for element in map_output]

# compare to using model.predict which merges the outputs from all images
predict_output = model.predict(ds_validation_roi.take(5))

<h2>Batch evaluation - model.evaluate() </h2>

In [ ]:
# performance evaluation on multiple images from a tf.data.Dataset
metrics = model.evaluate(ds_validation_roi)

<h2>Build Dataset from dictionary of instructions</h2>

In [ ]:
import copy
import itk
import math
import numpy as np
import operator
import random
import re
import tensorflow as tf


class FindResolutionForSlide:
    """A class that computes read parameters for slides.

    An instance of class FindResolutionForSlide is a callable that will
    add level, factor, number_pixel_rows_for_slide, and number_pixel_columns_for_slide fields to a
    slide dictionary.

    It inputs are filename (which is read from the slide dictionary),
    desired_magnification, and magnification_tolerance.

    """

    def __init__(self, study, desired_magnification, magnification_tolerance):
        """Sanity check the supplied parameters and store them for later use."""
        # Check values.
        if not ("version" in study and study["version"] == "version-1"):
            raise ValueError('study["version"] must exist and be equal to "version-1".')
        if not (isinstance(desired_magnification, (int, float)) and 0 < desired_magnification):
            raise ValueError(f"desired_magnification ({desired_magnification})" " must be a positive number")
        if not (isinstance(magnification_tolerance, (int, float)) and 0 <= magnification_tolerance <= 1):
            raise ValueError(f"magnification_tolerance ({magnification_tolerance})" " must be a value in [0, 1]")

        # Save values.
        self.desired_magnification = desired_magnification
        self.magnification_tolerance = magnification_tolerance

    def __call__(self, slide):
        """Add level, factor, number_pixel_rows_for_slide, and number_pixel_columns_for_slide fields to a
        slide dictionary.
        """
        # Check values.
        if "filename" not in slide:
            raise ValueError('slide["filename"] must be already set.')
        filename = slide["filename"]

        # Do the work.
        if re.compile(r"\.svs$").search(filename):
            import openslide as os

            # read whole-slide image file and create openslide object
            os_obj = os.OpenSlide(filename)

            # measure objective of level 0
            objective = np.float32(os_obj.properties[os.PROPERTY_NAME_OBJECTIVE_POWER])

            # calculate magnifications of levels
            estimated = np.array(objective / os_obj.level_downsamples)

            # Find best level to use and its factor
            level, factor = self._get_level_and_factor(
                self.desired_magnification, estimated, self.magnification_tolerance
            )

            # get slide number_pixel_columns_for_slide, number_pixel_rows_for_slide at desired
            # magnification. (Note number_pixel_columns_for_slide before number_pixel_rows_for_slide)
            number_pixel_columns_for_slide, number_pixel_rows_for_slide = os_obj.level_dimensions[level]

        elif re.compile(r"\.zarr$").search(filename):
            import zarr

            # read whole-slide image and create zarr objects
            store = zarr.DirectoryStore(filename)
            source_group = zarr.open(store, mode="r")

            # measure objective of level 0
            objective = np.float32(source_group.attrs[os.PROPERTY_NAME_OBJECTIVE_POWER])

            # calculate magnifications of levels
            estimated = np.array(objective / source_group.attrs["level_downsamples"])

            # Find best level to use and its factor
            level, factor = self._get_level_and_factor(
                self.desired_magnification, estimated, self.magnification_tolerance
            )

            # get slide number_pixel_columns_for_slide, number_pixel_rows_for_slide at desired
            # magnification. (Note number_pixel_rows_for_slide before number_pixel_columns_for_slide)
            number_pixel_rows_for_slide, number_pixel_columns_for_slide = source_group[format(level)].shape[0:2]

        else:
            from PIL import Image

            # We don't know magnifications so assume reasonable values for level and
            # factor.
            level = 0
            factor = 1.0
            pil_obj = Image.open(filename)
            number_pixel_columns_for_slide, number_pixel_rows_for_slide = pil_obj.size

        slide["level"] = level
        slide["factor"] = factor
        slide["number_pixel_rows_for_slide"] = number_pixel_rows_for_slide
        slide["number_pixel_columns_for_slide"] = number_pixel_columns_for_slide

    def _get_level_and_factor(self, desired_magnification, estimated, magnification_tolerance):
        """A private subroutine that computes level and factor."""
        # calculate difference with magnification levels
        delta = desired_magnification - estimated

        # match to existing levels
        if np.min(np.abs(np.divide(delta, desired_magnification))) < magnification_tolerance:  # match
            level = np.squeeze(np.argmin(np.abs(delta)))
            factor = 1.0
        elif np.any(delta < 0):
            value = np.max(delta[delta < 0])
            level = np.squeeze(np.argwhere(delta == value)[0])
            factor = desired_magnification / estimated[level]
        else:  # desired magnification above base level - throw error
            raise ValueError("Cannot interpolate above scan magnification.")

        return level, factor


class TilesByGridAndMask:
    """Select tiles according to a regular grid.  Optionally, restrict the list by a mask.
    Optionally, select a random subset of them.
    """

    def __init__(
        self,
        study,
        randomly_select=-1,  # Defaults to select all
        number_pixel_overlap_rows_for_tile=0,  # Defaults to no overlap between adjacent tiles
        number_pixel_overlap_columns_for_tile=0,
        mask_filename="",  # Defaults to no masking
    ):
        """Sanity check the supplied parameters and store them for later use."""
        # Check values.
        if not ("version" in study and study["version"] == "version-1"):
            raise ValueError('study["version"] must exist and be equal to "version-1".')
        if not (
            "number_pixel_rows_for_tile" in study
            and isinstance(study["number_pixel_rows_for_tile"], int)
            and study["number_pixel_rows_for_tile"] > 0
        ):
            raise ValueError('study["number_pixel_rows_for_tile"]' " must exist and be a positive integer")
        if not (
            "number_pixel_columns_for_tile" in study
            and isinstance(study["number_pixel_columns_for_tile"], int)
            and study["number_pixel_columns_for_tile"] > 0
        ):
            raise ValueError('study["number_pixel_columns_for_tile"]' " must exist and be a positive integer")
        if not (isinstance(randomly_select, int) and -1 <= randomly_select):
            raise ValueError(f"randomly_select ({randomly_select})" " must be a non-negative integer or -1.")
        if not (
            isinstance(number_pixel_overlap_rows_for_tile, int)
            and number_pixel_overlap_rows_for_tile < study["number_pixel_rows_for_tile"]
        ):
            raise ValueError(
                f"number_pixel_overlap_rows_for_tile ({number_pixel_overlap_rows_for_tile})"
                " must be less than"
                f' number_pixel_rows_for_tile ({study["number_pixel_rows_for_tile"]}).'
            )
        if not (
            isinstance(number_pixel_overlap_columns_for_tile, int)
            and number_pixel_overlap_columns_for_tile < study["number_pixel_columns_for_tile"]
        ):
            raise ValueError(
                f"number_pixel_overlap_columns_for_tile ({number_pixel_overlap_columns_for_tile})"
                " must be less than"
                f' number_pixel_columns_for_tile ({study["number_pixel_columns_for_tile"]}).'
            )
        if mask_filename != "":
            mask_itk = itk.imread(mask_filename)  # May throw exception
            if mask_itk.GetImageDimension() != 2:
                raise ValueError(f"The mask ({mask_filename})" " should be a 2-dimensional image.")

        # Save values.  To keep garbage collection efficient don't save all of `study`.
        self.number_pixel_rows_for_tile = study["number_pixel_rows_for_tile"]
        self.number_pixel_columns_for_tile = study["number_pixel_columns_for_tile"]
        self.randomly_select = randomly_select
        self.number_pixel_overlap_rows_for_tile = number_pixel_overlap_rows_for_tile
        self.number_pixel_overlap_columns_for_tile = number_pixel_overlap_columns_for_tile
        self.mask_filename = mask_filename
        if mask_filename != "":
            self.mask_itk = mask_itk

    def __call__(self, slide):
        """Select tiles according to a regular grid.  Optionally, restrict the list by a mask.
        Optionally, select a random subset of them.
        """
        # Check values.
        if "number_pixel_rows_for_slide" not in slide:
            raise ValueError('slide["number_pixel_rows_for_slide"] must be already set.')
        if "number_pixel_columns_for_slide" not in slide:
            raise ValueError('slide["number_pixel_columns_for_slide"] must be already set.')

        # Do the work.
        row_stride = self.number_pixel_rows_for_tile - self.number_pixel_overlap_rows_for_tile
        number_tile_rows_for_slide = slide["number_tile_rows_for_slide"] = math.floor(
            (slide["number_pixel_rows_for_slide"] - self.number_pixel_overlap_rows_for_tile) / row_stride
        )
        column_stride = self.number_pixel_columns_for_tile - self.number_pixel_overlap_columns_for_tile
        number_tile_columns_for_slide = slide["number_tile_columns_for_slide"] = math.floor(
            (slide["number_pixel_columns_for_slide"] - self.number_pixel_overlap_columns_for_tile) / column_stride
        )
        has_mask = hasattr(self, "mask_itk")
        if has_mask:
            # We will change the resolution of the mask (if
            # necessary), which will change the number of pixels, but
            # will not change the overall physical size represented by
            # the image nor the position of the upper left corner of
            # its upper left pixel.
            input_size = itk.size(self.mask_itk)
            output_size = [number_tile_columns_for_slide, number_tile_rows_for_slide]
            if input_size != output_size:
                # print(f"Resampling from input_size = {input_size} to output_size = {output_size}")
                # Check that the input and output aspect ratios are pretty close
                if abs(math.log((output_size[0] / input_size[0]) / (output_size[1] / input_size[1]))) > 0.20:
                    raise ValueError("The mask aspect ratio does not match that for the number of tiles.")
                input_spacing = itk.spacing(self.mask_itk)
                input_origin = itk.origin(self.mask_itk)
                image_dimension = self.mask_itk.GetImageDimension()
                output_spacing = [input_spacing[d] * input_size[d] / output_size[d] for d in range(image_dimension)]
                output_origin = [
                    input_origin[d] + 0.5 * (output_spacing[d] - input_spacing[d]) for d in range(image_dimension)
                ]
                interpolator = itk.NearestNeighborInterpolateImageFunction.New(self.mask_itk)
                resampled_mask_itk = itk.resample_image_filter(
                    self.mask_itk,
                    interpolator=interpolator,
                    size=output_size,
                    output_spacing=output_spacing,
                    output_origin=output_origin,
                )
            else:
                resampled_mask_itk = self.mask_itk

        tiles = slide["tiles"] = {}
        number_of_tiles = 0
        for row in range(number_tile_rows_for_slide):
            for column in range(number_tile_columns_for_slide):
                if not (has_mask and resampled_mask_itk[row, column] == 0):
                    tiles[f"tile_{number_of_tiles}"] = {
                        "tile_top": row * row_stride,
                        "tile_left": column * column_stride,
                    }
                number_of_tiles += 1  # Increment even if tile is skipped.
        # Choose a subset of the tiles randomly
        all_tile_names = tiles.keys()
        if 0 <= self.randomly_select < len(all_tile_names):
            keys_to_remove = random.sample(all_tile_names, len(all_tile_names) - self.randomly_select)
            for key in keys_to_remove:
                del tiles[key]


class TilesByList:
    """Select the tiles supplied by the user.  Optionally, select a random subset of them."""

    def __init__(
        self,
        study,
        randomly_select=-1,  # Defaults to select all
        tiles_dictionary={},  # {'AB234': {'tile_top': top0, 'tile_left': left0}, 'CD43': {'tile_top': top1, 'tile_left': left1}, ...}
    ):
        """Sanity check the supplied parameters and store them for later use."""
        # Check values
        if not ("version" in study and study["version"] == "version-1"):
            raise ValueError('study["version"] must exist and be equal to "version-1".')
        if not (
            "number_pixel_rows_for_tile" in study
            and isinstance(study["number_pixel_rows_for_tile"], int)
            and study["number_pixel_rows_for_tile"] > 0
        ):
            raise ValueError('study["number_pixel_rows_for_tile"]' " must exist and be a positive integer")
        if not (
            "number_pixel_columns_for_tile" in study
            and isinstance(study["number_pixel_columns_for_tile"], int)
            and study["number_pixel_columns_for_tile"] > 0
        ):
            raise ValueError('study["number_pixel_columns_for_tile"]' " must exist and be a positive integer")
        if not (isinstance(randomly_select, int) and -1 <= randomly_select):
            raise ValueError(f"randomly_select ({randomly_select})" " must be a non-negative integer or -1.")
        if not (
            isinstance(tiles_dictionary, dict)
            and all([isinstance(tile_corner, dict) for tile_corner in tiles_dictionary.values()])
            and all(
                [
                    key in tile_corner.keys()
                    for tile_corner in tiles_dictionary.values()
                    for key in ("tile_top", "tile_left")
                ]
            )
            and all(
                [
                    isinstance(tile_corner[key], int)
                    for tile_corner in tiles_dictionary.values()
                    for key in ("tile_top", "tile_left")
                ]
            )
            and all(
                [
                    tile_corner[key] >= 0
                    for tile_corner in tiles_dictionary.values()
                    for key in ("tile_top", "tile_left")
                ]
            )
        ):
            raise ValueError(
                "tiles_dictionary must be dictionary of tiles."
                '  Each tile is a dictionary, with keys "tile_top" and "tile_left"'
                " and with values that are non-negative integers."
            )

        # Save values.  To keep garbage collection efficient don't save all of `study`,
        # just the parts that we need.
        self.number_pixel_rows_for_tile = study["number_pixel_rows_for_tile"]
        self.number_pixel_columns_for_tile = study["number_pixel_columns_for_tile"]
        self.randomly_select = randomly_select
        self.tiles_dictionary = copy.deepcopy(tiles_dictionary)  # in case user changes it later

    def __call__(self, slide):
        """Select the tiles supplied by the user.  Optionally, select a random subset of them."""
        tiles = slide["tiles"] = copy.deepcopy(self.tiles_dictionary)  # in case __call__ is called again.
        all_tile_names = tiles.keys()
        if 0 <= self.randomly_select < len(all_tile_names):
            keys_to_remove = random.sample(all_tile_names, len(all_tile_names) - self.randomly_select)
            for key in keys_to_remove:
                del tiles[key]


class TilesRandomly:
    """Select a random subset of all possible tiles."""

    def __init__(
        self,
        study,
        randomly_select=1,  # Defaults to select one
    ):
        """Sanity check the supplied parameters and store them for later use."""
        # Check values.
        if not ("version" in study and study["version"] == "version-1"):
            raise ValueError('study["version"] must exist and be equal to "version-1".')
        if not (
            "number_pixel_rows_for_tile" in study
            and isinstance(study["number_pixel_rows_for_tile"], int)
            and study["number_pixel_rows_for_tile"] > 0
        ):
            raise ValueError('study["number_pixel_rows_for_tile"]' " must exist and be a positive integer")
        if not (
            "number_pixel_columns_for_tile" in study
            and isinstance(study["number_pixel_columns_for_tile"], int)
            and study["number_pixel_columns_for_tile"] > 0
        ):
            raise ValueError('study["number_pixel_columns_for_tile"]' " must exist and be a positive integer")
        if not (isinstance(randomly_select, int) and 0 <= randomly_select):
            raise ValueError(f"randomly_select ({randomly_select})" " must be a non-negative integer.")

        # Save values.  To keep garbage collection efficient don't save all of `study`.
        self.number_pixel_rows_for_tile = study["number_pixel_rows_for_tile"]
        self.number_pixel_columns_for_tile = study["number_pixel_columns_for_tile"]
        self.randomly_select = randomly_select

    def __call__(self, slide):
        """Select a random subset of all possible tiles."""
        if "number_pixel_rows_for_slide" not in slide:
            raise ValueError('slide["number_pixel_rows_for_slide"] must be already set.')
        if "number_pixel_columns_for_slide" not in slide:
            raise ValueError('slide["number_pixel_columns_for_slide"] must be already set.')

        row_too_big = slide["number_pixel_rows_for_slide"] - self.number_pixel_rows_for_tile + 1
        column_too_big = slide["number_pixel_columns_for_slide"] - self.number_pixel_columns_for_tile + 1
        row_column_list = [
            (random.randrange(0, row_too_big), random.randrange(0, column_too_big)) for _ in range(self.randomly_select)
        ]
        tiles = slide["tiles"] = {}
        number_of_tiles = 0
        for (row, column) in row_column_list:
            tiles[f"tile_{number_of_tiles}"] = {"tile_top": row, "tile_left": column}
            number_of_tiles += 1


class CreateTensorFlowDataset:
    def __init__(self):
        self.dataset_map_options = {
            "num_parallel_calls": tf.data.experimental.AUTOTUNE,
            "deterministic": False,
        }

    def __call__(self, study_description):
        """From scratch, creates a tensorflow dataset with one tensorflow element per tile"""

        if not ("version" in study_description and study_description["version"] == "version-1"):
            raise ValueError('study_description["version"] must exist and be equal to "version-1".')
        if not (
            "number_pixel_rows_for_tile" in study_description
            and isinstance(study_description["number_pixel_rows_for_tile"], int)
            and study_description["number_pixel_rows_for_tile"] > 0
        ):
            raise ValueError('study_description["number_pixel_rows_for_tile"]' " must exist and be a positive integer")
        if not (
            "number_pixel_columns_for_tile" in study_description
            and isinstance(study_description["number_pixel_columns_for_tile"], int)
            and study_description["number_pixel_columns_for_tile"] > 0
        ):
            raise ValueError(
                'study_description["number_pixel_columns_for_tile"]' " must exist and be a positive integer"
            )
        # Check that other necessary keys are also present!!!

        # Partition the set of tiles into chunks.
        self._designate_chunks_for_tiles(study_description)
        # cProfile.runctx("self._designate_chunks_for_tiles(study_description)", globals=globals(), locals=locals(), sort="cumulative")
        # print("_designate_chunks_for_tiles done")

        self.number_pixel_rows_for_tile = tf.convert_to_tensor(study_description["number_pixel_rows_for_tile"])
        self.number_pixel_columns_for_tile = tf.convert_to_tensor(study_description["number_pixel_columns_for_tile"])

        # Start converting our description into tensors.
        study_as_tensors = {
            study_key: [tf.convert_to_tensor(study_description[study_key])]
            for study_key in study_description.keys()
            if study_key != "slides"
        }
        # print("study_as_tensors done")

        number_of_chunks = 0
        for slide_description in study_description["slides"].values():
            slide_as_tensors = {
                **study_as_tensors,
                **{
                    slide_key: [tf.convert_to_tensor(slide_description[slide_key])]
                    for slide_key in slide_description.keys()
                    if slide_key != "chunks"
                },
            }

            for chunk_description in slide_description["chunks"].values():
                chunk_as_tensors = {
                    **slide_as_tensors,
                    **{
                        chunk_key: [tf.convert_to_tensor(chunk_description[chunk_key])]
                        for chunk_key in chunk_description.keys()
                        if chunk_key != "tiles"
                    },
                    "tiles_top": [
                        tf.convert_to_tensor([tile["tile_top"] for tile in chunk_description["tiles"].values()])
                    ],
                    "tiles_left": [
                        tf.convert_to_tensor([tile["tile_left"] for tile in chunk_description["tiles"].values()])
                    ],
                }

                # Make a tensorflow Dataset from this chunk.
                chunk_dataset = tf.data.Dataset.from_tensor_slices(chunk_as_tensors)
                if number_of_chunks == 0:
                    study_dataset = chunk_dataset
                else:
                    study_dataset = study_dataset.concatenate(chunk_dataset)
                number_of_chunks += 1

        # We have accumulated the chunk datasets into a study_dataset where each element is a chunk.  Read in the chunk
        # pixel data and split it into tiles.
        study_dataset = study_dataset.map(self._read_and_split_chunk_pixels, **self.dataset_map_options)
        # print("_read_and_split_chunk_pixels done")
        # Change study_dataset so that each element is a tile.
        study_dataset = study_dataset.unbatch()
        # print("unbatch done")
        # Make the tile pixels easier to find in each study_dataset element.
        study_dataset = study_dataset.map(lambda elem: (elem.pop("tile_pixels"), elem), **self.dataset_map_options)
        # print("pop done")

        return study_dataset

    def _designate_chunks_for_tiles(self, study_description):
        number_pixel_rows_for_tile = study_description["number_pixel_rows_for_tile"]
        number_pixel_columns_for_tile = study_description["number_pixel_columns_for_tile"]

        for slide in study_description["slides"].values():
            if not (
                "number_pixel_rows_for_chunk" in slide
                and isinstance(slide["number_pixel_rows_for_chunk"], int)
                and slide["number_pixel_rows_for_chunk"] > 0
            ):
                raise ValueError('slide["number_pixel_rows_for_chunk"]' " must exist and be a positive integer")
            if not (
                "number_pixel_columns_for_chunk" in slide
                and isinstance(slide["number_pixel_columns_for_chunk"], int)
                and slide["number_pixel_columns_for_chunk"] > 0
            ):
                raise ValueError('slide["number_pixel_columns_for_chunk"]' " must exist and be a positive integer")
            number_pixel_rows_for_chunk = slide["number_pixel_rows_for_chunk"]
            number_pixel_columns_for_chunk = slide["number_pixel_columns_for_chunk"]

            tiles_as_sorted_list = list(slide["tiles"].items())
            del slide["tiles"]
            tiles_as_sorted_list.sort(key=lambda x: x[1]["tile_left"])  # second priority key
            tiles_as_sorted_list.sort(key=lambda x: x[1]["tile_top"])  # first priority key
            chunks = slide["chunks"] = {}
            number_of_chunks = 0
            while len(tiles_as_sorted_list) > 0:
                tile = tiles_as_sorted_list[0]
                chunk = chunks[f"chunk_{number_of_chunks}"] = {
                    "chunk_top": tile[1]["tile_top"],
                    "chunk_left": tile[1]["tile_left"],
                    "chunk_bottom": tile[1]["tile_top"] + number_pixel_rows_for_chunk,
                    "chunk_right": tile[1]["tile_left"] + number_pixel_columns_for_chunk,
                }
                number_of_chunks += 1

                if True:
                    # This implementations has a run time that is quadratic in the number of tiles that a slide has.  It
                    # is too slow; we should make it faster.
                    tiles = chunk["tiles"] = {}
                    subsequent_chunks = []
                    for tile in tiles_as_sorted_list:
                        if (
                            tile[1]["tile_top"] + number_pixel_rows_for_tile <= chunk["chunk_bottom"]
                            and tile[1]["tile_left"] + number_pixel_columns_for_tile <= chunk["chunk_right"]
                            and tile[1]["tile_left"] >= chunk["chunk_left"]
                            and tile[1]["tile_top"] >= chunk["chunk_top"]
                        ):
                            tiles[tile[0]] = tile[1]
                        else:
                            subsequent_chunks.append(tile)

                else:
                    # This implementations has a run time that is quadratic in the number of tiles that a slide has.  It
                    # is even slower than the above.
                    tiles = chunk["tiles"] = {
                        tile[0]: tile[1]
                        for tile in tiles_as_sorted_list
                        if tile[1]["tile_top"] + number_pixel_rows_for_tile <= chunk["chunk_bottom"]
                        and tile[1]["tile_left"] + number_pixel_columns_for_tile <= chunk["chunk_right"]
                        and tile[1]["tile_left"] >= chunk["chunk_left"]
                        and tile[1]["tile_top"] >= chunk["chunk_top"]
                    }
                    subsequent_chunks = [
                        tile
                        for tile in tiles_as_sorted_list
                        if not (
                            tile[1]["tile_top"] + number_pixel_rows_for_tile <= chunk["chunk_bottom"]
                            and tile[1]["tile_left"] + number_pixel_columns_for_tile <= chunk["chunk_right"]
                            and tile[1]["tile_left"] >= chunk["chunk_left"]
                            and tile[1]["tile_top"] >= chunk["chunk_top"]
                        )
                    ]

                # Update the list of tiles that are not yet in chunks
                tiles_as_sorted_list = subsequent_chunks

                # Make the chunk as small as possible given the tiles that it must support.
                # Note that this also ensures that the pixels that are read do not run over
                # the bottom or right border of the slide (assuming that the tiles do not
                # go over those borders).
                chunk["chunk_top"] = min([tile["tile_top"] for tile in tiles.values()])
                chunk["chunk_left"] = min([tile["tile_left"] for tile in tiles.values()])
                chunk["chunk_bottom"] = max([tile["tile_top"] for tile in tiles.values()]) + number_pixel_rows_for_tile
                chunk["chunk_right"] = (
                    max([tile["tile_left"] for tile in tiles.values()]) + number_pixel_columns_for_tile
                )

    @tf.function
    def _read_and_split_chunk_pixels(self, elem):
        # Get chunk's pixel data from disk and load it into chunk_pixels_as_tensor
        chunk_pixels_as_tensor = tf.py_function(
            func=self._py_read_chunk_pixels,
            inp=[
                elem["chunk_top"],
                elem["chunk_left"],
                elem["chunk_bottom"],
                elem["chunk_right"],
                elem["filename"],
                elem["level"],
            ],
            Tout=tf.uint8,
        )
        number_of_tiles = tf.size(elem["tiles_top"])
        tiles = tf.TensorArray(dtype=tf.uint8, size=number_of_tiles)

        def condition(i, _):
            return tf.less(i, number_of_tiles)

        def body(i, tiles):
            return (
                i + 1,
                tiles.write(
                    i,
                    tf.image.crop_to_bounding_box(
                        chunk_pixels_as_tensor,
                        tf.gather(elem["tiles_top"], i) - elem["chunk_top"],
                        tf.gather(elem["tiles_left"], i) - elem["chunk_left"],
                        elem["number_pixel_rows_for_tile"],
                        elem["number_pixel_columns_for_tile"],
                    ),
                ),
            )

        _, tiles = tf.while_loop(condition, body, [0, tiles])
        tiles = tiles.stack()

        response = {}
        for key in elem.keys():
            if key not in ("tiles_top", "tiles_left"):
                response[key] = tf.repeat(elem[key], number_of_tiles)

        response = {**response, "tile_top": elem["tiles_top"], "tile_left": elem["tiles_left"], "tile_pixels": tiles}
        return response

    def _py_read_chunk_pixels(self, chunk_top, chunk_left, chunk_bottom, chunk_right, filename, level=-1):
        """Read from disk all the pixel data for a specific chunk of the whole slide."""

        filename = filename.numpy().decode("utf-8")
        chunk_top = chunk_top.numpy()
        chunk_left = chunk_left.numpy()
        chunk_bottom = chunk_bottom.numpy()
        chunk_right = chunk_right.numpy()
        level = level.numpy()

        if re.compile(r"\.svs$").search(filename):
            import openslide as os

            os_obj = os.OpenSlide(filename)
            chunk = np.array(
                os_obj.read_region((chunk_left, chunk_top), level, (chunk_right - chunk_left, chunk_bottom - chunk_top))
            )
        else:
            from PIL import Image

            pil_obj = Image.open(filename)
            chunk = np.asarray(pil_obj)[chunk_left:chunk_right, chunk_top:chunk_bottom, :]

        # Do we want to support other than RGB and/or other than uint8?!!!
        return tf.convert_to_tensor(chunk[..., :3], dtype=tf.uint8)


import copy

# Create a study and insert study-wide information
my_study0 = {"version": "version-1"}
my_study0["number_pixel_rows_for_tile"] = 256
my_study0["number_pixel_columns_for_tile"] = 256
my_slides = my_study0["slides"] = {}

# Add a slide to the study, including slide-wide information with it.
my_slide0 = my_slides["Slide_0"] = {}
my_slide0["filename"] = "/tf/notebooks/histomics_detect/example/DCBT_10_CMYC.svs"
# my_slide0["filename"] = "/tf/notebooks/histomics_stream/example/TCGA-BH-A0BZ-01Z-00-DX1.45EB3E93-A871-49C6-9EAE-90D98AE01913.svs"
my_slide0["slide_name"] = "DCBT_10_CMYC"
my_slide0["slide_group"] = "DCBT_10"
my_slide0["number_pixel_rows_for_chunk"] = 2048
my_slide0["number_pixel_columns_for_chunk"] = 2048

# For each slide, find the appropriate resolution given the desired_magnification and magnification_tolerance.  In this
# example, we use the same parameters for each slide, but this is not required generally.
find_resolution_for_slide = FindResolutionForSlide(my_study0, desired_magnification=20, magnification_tolerance=0.02)
for slide in my_study0["slides"].values():
    find_resolution_for_slide(slide)
print("================================================================")
print(f"my_study0 = {my_study0}")

# We are going to demonstrate several approaches to choosing tiles.  Each approach will start with its own copy of the
# my_study0 that we have built so far.

if True:
    # Demonstrate TilesByGridAndMask without a mask
    my_study_tiles_by_grid = copy.deepcopy(my_study0)
    tiles_by_grid = TilesByGridAndMask(
        my_study_tiles_by_grid,
        number_pixel_overlap_rows_for_tile=32,
        number_pixel_overlap_columns_for_tile=32,
        randomly_select=100,
    )
    # We could apply this to a subset of the slides, but we will apply it to all slides in this example.
    for slide in my_study_tiles_by_grid["slides"].values():
        tiles_by_grid(slide)
    # print("================================================================")
    print("Finished with TilesByGrid")
    # print(f"my_study_tiles_by_grid = {my_study_tiles_by_grid}")

if False:
    # Demonstrate TilesByGridAndMask with a mask
    my_study_tiles_by_grid_and_mask = copy.deepcopy(my_study0)
    tiles_by_grid_and_mask = TilesByGridAndMask(
        my_study_tiles_by_grid_and_mask,
        number_pixel_overlap_rows_for_tile=0,
        number_pixel_overlap_columns_for_tile=0,
        mask_filename="/tf/notebooks/histomics_stream/example/TCGA-BH-A0BZ-01Z-00-DX1.45EB3E93-A871-49C6-9EAE-90D98AE01913-mask.png",
        randomly_select=100,
    )
    # We could apply this to a subset of the slides, but we will apply it to all slides in this example.
    for slide in my_study_tiles_by_grid_and_mask["slides"].values():
        tiles_by_grid_and_mask(slide)
    # print("================================================================")
    print("Finished with TilesByGridAndMask")
    # print(f"my_study_tiles_by_grid_and_mask = {my_study_tiles_by_grid_and_mask}")

if True:
    # Demonstrate TilesByList
    my_study_tiles_by_list = copy.deepcopy(my_study0)
    tiles_by_list = TilesByList(
        my_study_tiles_by_list, randomly_select=5, tiles_dictionary=my_study_tiles_by_grid["slides"]["Slide_0"]["tiles"]
    )
    # We could apply this to a subset of the slides, but we will apply it to all slides in this example.
    for slide in my_study_tiles_by_list["slides"].values():
        tiles_by_list(slide)
    # print("================================================================")
    print("Finished with TilesByList")
    # print(f"my_study_tiles_by_list = {my_study_tiles_by_list}")

if True:
    # Demonstrate TilesRandomly
    my_study_tiles_randomly = copy.deepcopy(my_study0)
    tiles_randomly = TilesRandomly(my_study_tiles_randomly, randomly_select=3)
    # We could apply this to a subset of the slides, but we will apply it to all slides in this example.
    for slide in my_study_tiles_randomly["slides"].values():
        tiles_randomly(slide)
    # print("================================================================")
    print("Finished with TilesRandomly")
    # print(f"my_study_tiles_randomly = {my_study_tiles_randomly}")

# We choose one of the above examples for further processing.
my_study_of_tiles = my_study_tiles_by_grid
# my_study_of_tiles = my_study_tiles_randomly

create_tensorflow_dataset = CreateTensorFlowDataset()
tiles = create_tensorflow_dataset(my_study_of_tiles)
print("Finished with CreateTensorFlowDataset")

# print("================================================================")
# print(tiles)
# print("================================================================")
# tf.print(tiles)


<h3>Run with the tiles dataset</h3>

In [ ]:
dataset_map_options = {
    "num_parallel_calls": tf.data.experimental.AUTOTUNE,
    "deterministic": False,
}

# Convert pixel data to uint8
tiles = tiles.map(lambda x, y: (tf.cast(x, tf.uint8), y), **dataset_map_options)

# Run the model on the tiles
tiles = tiles.map(lambda x, y: (x, model(x, tau=0.5, nms_iou=0.3), y), **dataset_map_options)

<h3>Find a tile with many detections</h3>

In [ ]:
tiles2 = tiles
tiles2 = tiles2.map(lambda x, p, y: (x, p, tf.shape(p)[0], y), **dataset_map_options)

max_number_detections = -1
number_tiles = 0
for tile in tiles2:
    number_tiles = number_tiles + 1
    rgb, regressions, number_detections, _ = tile
    if number_detections > max_number_detections:
        tf.print(f"New best: Tile #{number_tiles} has {number_detections} detections.")
        max_number_detections = number_detections
        max_rgb = rgb
        max_regressions = regressions
    if max_number_detections ** 2 * number_tiles >= 10000:
        break
tf.print(f"Examined {number_tiles} tiles in total.")
plot_inference(max_rgb, max_regressions)

<h2>Save and Load Model Weights</h2>

In [ ]:
# save checkpoint
model.save_weights("/tf/notebooks/histomics_detect/example/saved_model/")

# create dummy network for restore
restored = FasterRCNN(rpnetwork, backbone, [width, height], anchor_px, rpn_lmbda)
restored.load_weights("/tf/notebooks/histomics_detect/example/saved_model/")

# check that outputs are same
assert tf.math.reduce_all(tf.math.equal(restored(rgb), model(rgb)))